In [128]:
from copy import deepcopy
import subprocess
import socket

import sys

import concurrent.futures
import time

In [129]:
l,b,n=100,100,100
while l>9 or b>9: 
    l = int(input("Horizontal number of wifi-access points(<9): "))
    b = int(input("Vertical number of wifi-access points(<9): "))
n = 0
while n % 2 == 0 or n > 9:
    n = int(
        input("Number of mesh-points between two wifi-access points (odd and <=9): ")
    )
hops = int(input("Max hops: "))
n_threads = int(input("Enter number of threads: "))

Horizontal number of wifi-access points(<9): 4
Vertical number of wifi-access points(<9): 3
Number of mesh-points between two wifi-access points (odd and <=9): 7
Max hops: 5
Enter number of threads: 3


In [130]:
def n_num_in_list(_list, n):
    ans = []
    def n_num_in_list_util(_l, _n, _ans):
        if _n == 0:
            ans.append(_ans)
            return

        for i in range(len(_l)):
            a = deepcopy(_l)[i+1:]
            # a.pop(i)
            b = deepcopy(_ans)
            b.append(_l[i])
            n_num_in_list_util(
                a,
                _n-1,
                b    
            )

    n_num_in_list_util(_list, n, [])
    
    return ans

In [131]:
def conn_wifi_mesh(wifi_name, mesh_name):
    ans = False
    if int(mesh_name[-1]) <= int(n / 2):
        ans = ans or wifi_name == mesh_name[:2]
    if int(mesh_name[-1]) >= int(n / 2):
        ans = ans or wifi_name == mesh_name[2:4]
    return ans

def conn_mesh_mesh(m_name1, m_name2):
    fh1 = m_name1[:2]
    fh2 = m_name2[:2]

    lh1 = m_name1[2:4]
    lh2 = m_name2[2:4]

    if fh1 == fh2 and lh1 == lh2:
        return abs(int(m_name1[-1]) - int(m_name2[-1])) == 1

    if fh1 == fh2:
        return m_name1[-1] == 0 and m_name2[-1] == 0

    if fh1 == lh2:
        return m_name1[-1] == 0 and m_name2[-1] == n - 1
    if fh2 == lh1:
        return m_name2[-1] == 0 and m_name1[-1] == n - 1

    return False

In [132]:
def all_conn_mesh(mesh, key):
    res = set()
    for k in mesh.keys():
        if conn_mesh_mesh(k, key):
            res.add(k)
    return res

In [133]:
# param = {'00':True, '01':False, ...}
def total_connected_devices_util(wifi={}):
    mesh = {}

    pi, pj = None, None
    for i in range(b):
        for j in range(l):
            if pi != None and pj != None:
                for k in range(n):
                    mesh[str(pi) + str(pj) + str(i) + str(j) + str(k)] = False
            pj = j
            pi = i
        pj = None

    pi, pj = None, None
    for j in range(l):
        for i in range(b):
            if pi != None and pj != None:
                for k in range(n):
                    mesh[str(pi) + str(pj) + str(i) + str(j) + str(k)] = False
            pi = i
            pj = j
        pi = None

    for i in range(b):
        for j in range(l):
            if str(i) + str(j) not in wifi:
                wifi[str(i) + str(j)] = True

    for key in mesh.keys():
        for k in wifi:
            if conn_wifi_mesh(k, key):
                mesh[key] = mesh[key] or wifi[k]

    tmp_keys = set([1])
    for key in mesh.keys():

        def fun(_key, _hops):
            if _hops > 0:
                for k in all_conn_mesh(mesh, _key):
                    tmp_keys.add(k)
                    fun(k, _hops - 1)

        if mesh[key]:
            fun(key, hops)
    for k in tmp_keys:
        mesh[k] = True

    # print(mesh)
    ct = 0
    for k, v in mesh.items():
        if v:
            ct += 1
        # else:
        #     print(v)
    return ct

In [134]:
def _local(off_indices, k, wifi):
    dic = {}
    for _ in off_indices:
        if _ !='':
            dic[wifi[int(_)]] = False
    return k * total_connected_devices_util(dic)

# pass number of wifi to turn off
def total_connected_devices(off_ct):
    wifi = []
    for i in range(b):
        for j in range(l):
            wifi.append(str(i) + str(j))
    
    off_indices_list = n_num_in_list([i for i in range(len(wifi))], off_ct)

    val, ct = 0, len(off_indices_list)

    # consider equivalent positions
    d_off_indices_list = {}
    for _list in off_indices_list:
        # Case 1
        l1 = []
        for _index in _list:
            ij = wifi[_index]
            i = b-1-int(ij[0])
            l1.append(wifi.index(str(i)+ij[1]))
        l1 = sorted(l1)    
        
        # Case 2
        l2 = []
        for _index in _list:
            ij = wifi[_index]
            j = l-1-int(ij[1])
            l2.append(wifi.index(ij[0] + str(j)))
        l2 = sorted(l2)
        
        # Case 3
        l3 = []
        for _index in _list:
            ij = wifi[_index]
            i = b-1-int(ij[0])
            j = l-1-int(ij[1])
#             print(ij,str(i)+str(j))
            l3.append(wifi.index(str(i)+str(j)))
        l3 = (sorted(l3))
        
#         print(l1,l2,l3)
            
        none_in_dic = True
        for _l in [l1,l2,l3]:
            if str(_l) in d_off_indices_list:
                d_off_indices_list[str(_l)] = d_off_indices_list[str(_l)]+1
                none_in_dic = False
                break
            
        
        if none_in_dic:
            d_off_indices_list[str(_list)]=1
#     print(d_off_indices_list)
                
    with concurrent.futures.ProcessPoolExecutor(max_workers=n_threads) as e:
        futures = [e.submit(_local, _list_str.strip('][').split(', '), d_off_indices_list[_list_str], wifi) for _list_str in d_off_indices_list.keys()]
        for f in futures: 
            if f.result():
                val+=f.result()
                
    return int(val/ct)

In [135]:
start = time.time()
_h = hops
# 0 hops -> only wifi
hops = 0
for ct in range(l*b):
    print(ct, total_connected_devices(off_ct = ct))
    
end = time.time()
print("\n\nTime taken: ", end-start, '\n\n\n')

0 120
1 111
2 102
3 93
4 84
5 74
6 65
7 55
8 44
9 34
10 23
11 12
Time taken:  2.035003185272217


In [136]:
start = time.time()
# lets say 5 hops allowed
hops = _h

for ct in range(l*b):
    print(ct, total_connected_devices(off_ct = ct))
    
end = time.time()
print("Time taken: ", end-start)

0 120
1 120
2 118
3 114
4 109
5 101
6 92
7 82
8 69
9 55
10 38
11 20
Time taken:  73.88578152656555


In [137]:
# def _local(off_indices, wifi):
#     dic = {}
#     for _ in off_indices:
#         if _ !='':
#             dic[wifi[int(_)]] = False
#     return total_connected_devices_util(dic)

# # pass number of wifi to turn off
# def total_connected_devices(off_ct):
#     wifi = []
#     for i in range(b):
#         for j in range(l):
#             wifi.append(str(i) + str(j))
    
#     off_indices_list = n_num_in_list([i for i in range(len(wifi))], off_ct)

#     val, ct = 0, len(off_indices_list)

                
#     with concurrent.futures.ProcessPoolExecutor(max_workers=n_threads) as e:
#         futures = [e.submit(_local, _list, wifi) for _list in off_indices_list]
#         for f in futures: 
#             if f.result():
#                 val+=f.result()
                
#     return int(val/ct)









# start = time.time()
# _h = hops
# # 0 hops -> only wifi
# hops = 0
# for ct in range(l*b):
#     print(ct, total_connected_devices(off_ct = ct))
# end = time.time()
# print("Time taken: ", end-start)
    
    
# start = time.time()
# # lets say 5 hops allowed
# hops = _h

# for ct in range(l*b):
#     print(ct, total_connected_devices(off_ct = ct))
# end = time.time()
# print("Time taken: ", end-start)

0 120
1 111
2 102
3 93
4 84
5 74
6 65
7 55
8 44
9 34
10 23
11 12
Time taken:  6.500740051269531
0 120
1 120
2 118
3 114
4 109
5 101
6 92
7 82
8 69
9 55
10 38
11 20
Time taken:  256.21805238723755
